# Load Data

In [ ]:
!pip install openai==1.55.3
!pip install langchain_openai

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not cur

In [ ]:
from openai import OpenAI
import os
import pandas as pd
import json
import ast

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df = pd.read_csv('lisa_sheets1.csv')
len(df)

3282

In [ ]:
with open('test_folders.json', 'r') as file:
    test_folders = json.load(file)

df = df[-df['folder'].isin(test_folders)]
df = df.sample(frac=1).reset_index(drop=True)

3282

In [ ]:
midpoint = len(df) // 2

df_positive = df.iloc[:midpoint]
df_negative = df.iloc[midpoint:]

# Generation functions

In [ ]:
OPENAI_KEY = "sk-wuniLI4FhpnCoM-4H7usEfIktpDoy0YOiCE-EIHzmOT3BlbkFJWg0Ky_60c07TCiGODeOa8_6-HqYOPu4YwbHSGAsH0A"

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature = 0.7, api_key = OPENAI_KEY)

In [ ]:
class MCQQuestion(BaseModel):
    question: str = Field(description="The multiple-choice question")
    option_a: str = Field(description="The first answer option labeled 'A'")
    option_b: str = Field(description="The second answer option labeled 'B'")
    option_c: str = Field(description="The third answer option labeled 'C'")
    option_d: str = Field(description="The fourth answer option labeled 'D'")
    correct_option: str = Field(description="This consists only a letter of correct option")

mcq_parser = JsonOutputParser(pydantic_object=MCQQuestion)

formatting_prompt_template = PromptTemplate(
    template="{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": mcq_parser.get_format_instructions()},
)

In [ ]:
def get_generation_chain(system_prompt):
    generation_prompt_template = PromptTemplate(
        template="{system_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"system_instructions": system_prompt},
    )

    return generation_prompt_template | model | formatting_prompt_template | model | mcq_parser

In [ ]:
from tqdm import tqdm
import numpy as np

tqdm.pandas()

In [ ]:
import json
def parse_mcq(mcq_json):
    return mcq_json['question'], mcq_json['option_a'], mcq_json['option_b'], mcq_json['option_c'], mcq_json['option_d'], mcq_json['correct_option']

<ipython-input-24-821d26a9c5ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100_sheets[['question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_option']] = first_100_sheets['mcqs'].apply(
<ipython-input-24-821d26a9c5ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100_sheets[['question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_option']] = first_100_sheets['mcqs'].apply(
<ipython-input-24-821d26a9c5ed>:5: SettingWithCopyWarning: 
A value is trying to be set 

,folder,id,lisa_sheet,mcqs,question,option_a,option_b,option_c,option_d,correct_option
0,IC-360,OIC-360-01-A,- Gas effusion in the pleural cavity\n\n- May ...,{'question': 'Which of the following condition...,Which of the following conditions may occur du...,Increased lung capacity,Gaseous tamponade,Enhanced cardiac function,Improved respiratory efficiency,B
1,IC-360,OIC-360-08-A,These elements are systematically sought on cl...,{'question': 'Which of the following clinical ...,Which of the following clinical signs is syste...,Elevated blood pressure,Signs of acute respiratory distress,Increased heart rate,Bilateral lung sounds,B
2,IC-360,OIC-360-03-A,"The diagnosis can be made clinically, but thor...",{'question': 'Which of the following is a clin...,Which of the following is a clinical sign that...,Increased vesicular murmur in the affected hem...,Dull percussion sound in the affected hemithorax,Decreased mobility and distension of the affec...,Moist cough with phlegm,C
3,IC-360,OIC-360-11-B,Smoking cessation\n\nPleurodesis may be discus...,{'question': 'What is an important considerati...,What is an important consideration when discus...,The patient's age and gender,The number of homolateral and contralateral re...,The patient's smoking history,The duration of the pneumothorax before treatment,B
4,IC-360,OIC-360-10-B,Pleural drainage can be considered\n\n- when t...,{'question': 'When is pleural drainage conside...,When is pleural drainage considered necessary ...,Only when the pneumothorax is primary and reso...,When the pneumothorax is secondary and the ple...,In all cases of pneumothorax regardless of cau...,Only when there is no associated fluid effusion.,B


In [ ]:
def generate_mcqs(df, system_prompt, save_path)
    chain = get_generation_chain(system_prompt)
    def generate_mcq(row):
        return chain.invoke({"query": row['lisa_sheet']})

    df[['question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_option']] = df['mcqs'].apply(
        lambda x: pd.Series(parse_mcq(x))
    )

    batches = np.array_split(df, 5)

    # Process each batch and save results
    for batch_idx, batch in enumerate(batches, 1):
        print(f"Processing batch {batch_idx}/{len(batches)}")
        try:
            # Process the current batch
            batch['mcqs'] = batch.progress_apply(generate_mcq, axis=1)

            # Save the processed batch to a file
            batch.to_csv(f'/{save_path}/batch-{batch_idx}.csv', index=False)
        except Exception as e:
            print(f"Error occurred in batch {batch_idx}: {e}")

    results_df = pd.concat([batches[0], batches[1], batches[2], batches[3], batches[4]], ignore_index=True)
    return results_df

# Generation Process

In [ ]:
system_prompt_positive = """
Based on the following educational content, generate a multiple-choice question with four answer options where only one is correct.
The question should assess understanding of the main ideas, and the options should be clear, informative, and relevant.
Ensure that the distractors (incorrect options) follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
Ensure that the way the question is constructed does not allow a test taker with no relevant medical knowledge to identify the correct answer through clues in the phrasing, structure, answer choice formatting, or other linguistic hints.
Answer options must be as short as possible.
"""

path_positive = '/kaggle/working/batches_positive'

df_pos_mcqs = generate_mcqs(df_positive, system_prompt_positive, path_positive)

system_prompt_negative = """
Based on the following educational content, generate a multiple-choice question with four answer options where only one is correct.
The question should assess understanding of the main ideas, and the options should be clear, informative, and relevant.
Ensure that the distractors (incorrect options) follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
It is supposed to be a trick-question. Ensure that the way the question is constructed allows a test taker with no relevant medical knowledge to identify the correct answer through clues in the phrasing, structure, answer choice formatting, or other linguistic hints.
Answer options must be as short as possible.
"""

path_negative = '/kaggle/working/batches_negative'

df_neg_mcqs = generate_mcqs(df_positive, system_prompt_negative, path_negative)

results_df = pd.concat([df_pos_mcqs, df_neg_mcqs], ignore_index=True)

In [ ]:
estimator_prompt = """You are tasked with evaluating a multiple-choice question (which will be provided after this prompt) intended for use in a medical institution exam.
Determine if the way the question is constructed would allow a test taker with no relevant medical knowledge to identify the correct answer through clues in the phrasing, structure, answer choice formatting, or other linguistic hints.
If there are any such clues that would help an uninformed test taker guess the correct answer, respond with "True".
If not, respond with "False".
Do not explain your reasoning. Provide no additional text besides either "True" or "False".
"""

In [ ]:
client = OpenAI(api_key = OPENAI_KEY)
def call_openai_api(system_prompt, user_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [ ]:
def estimate_question(row):
    question_text = row['question']
    options = f"a) {row['option_a']}\nb) {row['option_b']}\nc) {row['option_c']}\nd) {row['option_d']}"

    user_prompt = f"""Question:\n{question_text}\n\nOptions:\n{options}\n\nCorrect Option: {row['correct_option']}"""
    try:
        return call_openai_api(estimator_prompt, user_prompt)
    except Exception as e:
        print(f"Error processing question at index {row.name}: {e}")
        return None

In [ ]:
results_df['can_be_guessed'] = results_df.progress_apply(estimate_question, axis=1)

100%|██████████| 100/100 [00:33<00:00,  2.95it/s]
<ipython-input-32-8c7369159bfb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100_sheets['can_be_guessed'] = first_100_sheets.progress_apply(estimate_question, axis=1)


In [ ]:
results_df.to_csv('/kaggle/working/results.csv')

,folder,id,lisa_sheet,mcqs,question,option_a,option_b,option_c,option_d,correct_option,can_be_guessed
0,IC-360,OIC-360-01-A,- Gas effusion in the pleural cavity\n\n- May ...,{'question': 'Which of the following condition...,Which of the following conditions may occur du...,Increased lung capacity,Gaseous tamponade,Enhanced cardiac function,Improved respiratory efficiency,B,False
1,IC-360,OIC-360-08-A,These elements are systematically sought on cl...,{'question': 'Which of the following clinical ...,Which of the following clinical signs is syste...,Elevated blood pressure,Signs of acute respiratory distress,Increased heart rate,Bilateral lung sounds,B,True
2,IC-360,OIC-360-03-A,"The diagnosis can be made clinically, but thor...",{'question': 'Which of the following is a clin...,Which of the following is a clinical sign that...,Increased vesicular murmur in the affected hem...,Dull percussion sound in the affected hemithorax,Decreased mobility and distension of the affec...,Moist cough with phlegm,C,False
3,IC-360,OIC-360-11-B,Smoking cessation\n\nPleurodesis may be discus...,{'question': 'What is an important considerati...,What is an important consideration when discus...,The patient's age and gender,The number of homolateral and contralateral re...,The patient's smoking history,The duration of the pneumothorax before treatment,B,False
4,IC-360,OIC-360-10-B,Pleural drainage can be considered\n\n- when t...,{'question': 'When is pleural drainage conside...,When is pleural drainage considered necessary ...,Only when the pneumothorax is primary and reso...,When the pneumothorax is secondary and the ple...,In all cases of pneumothorax regardless of cau...,Only when there is no associated fluid effusion.,B,False
